In [0]:
import requests
import json

In [0]:
container_name = 'dropzone'
storage_account_name = 'a360c2x2555dz'
# secret_value = dbutils.secrets.get(scope='dbScope', key= 'secret')

# List of required secrets
required_secrets = [
    "ARIAB-SAS-TOKEN", "ARIAFTA-SAS-TOKEN", "ARIAJR-SAS-TOKEN",
    "ARIASB-SAS-TOKEN", "ARIATD-SAS-TOKEN", "ARIAUTA-SAS-TOKEN"
]

In [0]:
display(dbutils.fs.ls("/mnt/dropzoneariab/ARIAB/submission"))

In [0]:
response_files = dbutils.fs.ls("/mnt/dropzoneariab/ARIAB/response")
file_count = len(response_files)
file_count

In [0]:
test_message_df = spark.read.json("dbfs:/mnt/dropzoneariab/ARIAB/submission/test_message.json")
display(test_message_df)

In [0]:
dbutils.secrets.listScopes()

In [0]:
dbutils.secrets.list(scope='dbScope')

In [0]:
dbutils.secrets.list(scope='ingest00-meta002-sbox')

In [0]:
foldername = secret.split('-')[0]

In [0]:
for secret in required_secrets:
    foldername = secret.split('-')[0]
    mount_point = f"/mnt/{container_name}{foldername.lower()}"

    # Ensure mount_info is accessed correctly
    mounts = dbutils.fs.mounts()
    
    if not any(mount_point in (mount_info.mountPoint if isinstance(mount_info, tuple) else mount_info['mountPoint']) for mount_info in mounts):
        try:
            dbutils.fs.mount(
                source=source_url,
                mount_point=mount_point,
                extra_configs={f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": dbutils.secrets.get(scope='dbScope', key=secret)}
            )
            print(f"Container '{container_name}' mounted successfully at '{mount_point}'")
        except Exception as e:
            print(f"Error mounting container at '{mount_point}': {e}")
    else:
        print(f"Mount point '{mount_point}' already exists.")

    # Access the specific folder
    folder_path = f"{mount_point}/{foldername}"

    # List files in the folder to verify
    try:
        files = dbutils.fs.ls(folder_path)
        display(files)
        print(f"Folder '{foldername}' accessed successfully at '{folder_path}'")
    except Exception as e:
        print(f"Error accessing folder '{foldername}': {e}")


In [0]:
display(dbutils.fs.mounts())

In [0]:
dbutils.notebook.exit("Success")

## APPENDIX

### Use the code below to add or delete keys in the DB secret scope.

You need to obtain the databricks_token, databricks_instance, and keyvault_scope.

In [0]:

# Databricks Configurations
databricks_instance = "adb-3635282203417052.12.azuredatabricks.net"
databricks_token = "xxxxxxxx"  
db_scope = "dbScope"
keyvault_scope = "ingest00-keyvault-sbox"

# List of required secrets
required_secrets = [
    "ARIAB-SAS-TOKEN", "ARIAFTA-SAS-TOKEN", "ARIAJR-SAS-TOKEN",
    "ARIASB-SAS-TOKEN", "ARIATD-SAS-TOKEN", "ARIAUTA-SAS-TOKEN", "ingest00-metadata-mysql-username-sbox"
]

# Headers for API requests
headers = {
    "Authorization": f"Bearer {databricks_token}",
    "Content-Type": "application/json"
}

# Step 1: Get the existing secrets in `dbScope`
list_secrets_url = f"https://{databricks_instance}/api/2.0/secrets/list"
response = requests.get(list_secrets_url, headers=headers, json={"scope": db_scope})

if response.status_code == 200:
    existing_secrets = {secret["key"] for secret in response.json().get("secrets", [])}
else:
    print("Error fetching existing secrets:", response.text)
    existing_secrets = set()

# Step 2: Loop through required secrets and create missing ones
for secret_key in required_secrets:
    if secret_key not in existing_secrets:
        print(f"{secret_key} is missing. Fetching from {keyvault_scope}...")

        # Fetch secret from Azure Key Vault scope
        try:
            secret_value = dbutils.secrets.get(scope=keyvault_scope, key=secret_key)
        except Exception as e:
            print(f"Error fetching {secret_key} from {keyvault_scope}: {str(e)}")
            continue  # Skip if secret is not found in Key Vault

        # Step 3: Create the missing secret in `dbScope`
        create_secret_url = f"https://{databricks_instance}/api/2.0/secrets/put"
        payload = {
            "scope": db_scope,
            "key": secret_key,
            "string_value": secret_value
        }

        create_response = requests.post(create_secret_url, headers=headers, json=payload)

        if create_response.status_code == 200:
            print(f"Successfully added {secret_key} to {db_scope}.")
        else:
            print(f"Failed to add {secret_key}: {create_response.text}")
    else:
        print(f"{secret_key} already exists in {db_scope}.")


In [0]:

# # Databricks Configurations
# databricks_instance = "adb-3635282203417052.12.azuredatabricks.net"
# databricks_token = ""  
# db_scope = "dbScope"

# # List of secrets to delete
# secrets_to_delete = [
#     "ARIAB-SAS-TOKEN", "ARIAFTA-SAS-TOKEN", "ARIAJR-SAS-TOKEN",
#     "ARIASB-SAS-TOKEN", "ARIATD-SAS-TOKEN", "ARIAUTA-SAS-TOKEN",
#     "ingest00-metadata-mysql-username-sbox"
# ]

# # Headers for API requests
# headers = {
#     "Authorization": f"Bearer {databricks_token}",
#     "Content-Type": "application/json"
# }

# # Step 1: Get existing secrets in `dbScope`
# list_secrets_url = f"https://{databricks_instance}/api/2.0/secrets/list"
# response = requests.get(list_secrets_url, headers=headers, json={"scope": db_scope})

# if response.status_code == 200:
#     existing_secrets = {secret["key"] for secret in response.json().get("secrets", [])}
# else:
#     print("Error fetching existing secrets:", response.text)
#     existing_secrets = set()

# # Step 2: Delete secrets if they exist in `dbScope`
# for secret_key in secrets_to_delete:
#     if secret_key in existing_secrets:
#         delete_secret_url = f"https://{databricks_instance}/api/2.0/secrets/delete"
#         payload = {
#             "scope": db_scope,
#             "key": secret_key
#         }

#         delete_response = requests.post(delete_secret_url, headers=headers, json=payload)

#         if delete_response.status_code == 200:
#             print(f"Successfully deleted {secret_key} from {db_scope}.")
#         else:
#             print(f"Failed to delete {secret_key}: {delete_response.text}")
#     else:
#         print(f"{secret_key} not found in {db_scope}. Skipping deletion.")


In [0]:
dbutils.secrets.list(scope = 'dbScope')

In [0]:
dbutils.fs.ls('/mnt/dropzoneariasb/ARIASB/submission')

In [0]:
container_name = 'dropzone'
storage_account_name = 'a360c2x2555dz'
folder_name = 'ARIAJR'
sas_token = dbutils.secrets.get(scope='dbScope', key='ARIAJR-SAS-TOKEN')

In [0]:
sas_token = ""

spark.conf.set(
    f"fs.azure.sas.gold.ingest00curatedsbox.blob.core.windows.net",
    sas_token
)


In [0]:
df = spark.read.format("delta").load("wasbs://gold@ingest00curatedsbox.blob.core.windows.net/ARIADM/ARM/APPEALS/gold_appeals_with_a360/")
display(df)


In [0]:
sas_token = ""

spark.conf.set(
    f"fs.azure.sas.gold.ingest00curatedsbox.dfs.core.windows.net",
    sas_token
)

df = spark.read.format("delta").load("abfss://gold@ingest00curatedsbox.dfs.core.windows.net/ARIADM/ARM/APPEALS/gold_appeals_with_a360/")
display(df)

In [0]:
%python
sas_token = ""

spark.conf.set(
    "fs.azure.account.auth.type.ingest00curatedsbox.dfs.core.windows.net",
    "SAS"
)
spark.conf.set(
    "fs.azure.sas.ingest00curatedsbox.dfs.core.windows.net",
    sas_token
)

df = spark.read.format("delta").load("abfss://gold@ingest00curatedsbox.dfs.core.windows.net/ARIADM/ARM/APPEALS/gold_appeals_with_a360/")
display(df)

In [0]:
df = spark.read.format("delta").load("abfss://gold@ingest00curatedsbox.dfs.core.windows.net/ARIADM/ARM/APPEALS/gold_appeals_with_a360/")
display(df)



# Authentication Method Table
<table>
  <tr>
    <th>Authentication Method</th>
    <th>Spark Config Command</th>
    <th>Notes</th>
  </tr>
  <tr>
    <td><b>Storage Account Access Key</b></td>
    <td>fs.azure.account.key.ingest00curatedsbox.dfs.core.windows.net = &lt;access_key&gt;</td>
    <td>Simple method, but not recommended for security reasons.</td>
  </tr>
  <tr>
    <td><b>SAS Token (Recommended)</b></td>
    <td>fs.azure.sas.gold.ingest00curatedsbox.dfs.core.windows.net = &lt;sas_token&gt;</td>
    <td>More secure than access keys, grants scoped access.</td>
  </tr>
  <tr>
    <td><b>Service Principal (OAuth)</b></td>
    <td>
      fs.azure.account.auth.type.ingest00curatedsbox.dfs.core.windows.net = OAuth<br>
      fs.azure.account.oauth.provider.type.ingest00curatedsbox.dfs.core.windows.net = org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider<br>
      fs.azure.account.oauth2.client.id.ingest00curatedsbox.dfs.core.windows.net = &lt;client_id&gt;<br>
      fs.azure.account.oauth2.client.secret.ingest00curatedsbox.dfs.core.windows.net = &lt;client_secret&gt;<br>
      fs.azure.account.oauth2.client.endpoint.ingest00curatedsbox.dfs.core.windows.net = https://login.microsoftonline.com/&lt;tenant_id&gt;/oauth2/token
    </td>
    <td>Most secure, requires Azure AD Service Principal setup.</td>
  </tr>
  <tr>
    <td><b>Managed Identity (MSI)</b></td>
    <td>
      fs.azure.account.auth.type.ingest00curatedsbox.dfs.core.windows.net = OAuth<br>
      fs.azure.account.oauth.provider.type.ingest00curatedsbox.dfs.core.windows.net = org.apache.hadoop.fs.azurebfs.oauth2.MsiTokenProvider
    </td>
    <td>Best for Databricks on Azure, removes need for secrets.</td>
  </tr>
</table>


In [0]:
# Solution: Set Up OAuth for abfss:// Access
# Databricks needs an Azure AD Service Principal for authentication when using abfss://.

In [0]:
# Step 1: Retrieve Secrets for Authentication
# Ensure you have Client ID, Secret, and Tenant ID stored in Databricks Secrets (dbScope).

In [0]:
client_id = dbutils.secrets.get(scope="dbScope", key="client-id")
client_secret = dbutils.secrets.get(scope="dbScope", key="client-secret")
tenant_id = dbutils.secrets.get(scope="dbScope", key="tenant-id")


In [0]:
# Step 2: Configure Spark for abfss://
# Set up authentication for abfss://gold@ingest00curatedsbox.dfs.core.windows.net:

In [0]:
spark.conf.set("fs.azure.account.auth.type.ingest00curatedsbox.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.ingest00curatedsbox.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.ingest00curatedsbox.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.ingest00curatedsbox.dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.ingest00curatedsbox.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


In [0]:
sas_token = ""

spark.conf.set("fs.azure.sas.gold.ingest00curatedsbox.dfs.core.windows.net", sas_token)

df = spark.read.format("delta").load("abfss://gold@ingest00curatedsbox.dfs.core.windows.net/ARIADM/ARM/APPEALS/gold_appeals_with_a360/")

